<a href="https://colab.research.google.com/github/BelideSaiTeja/Fake-News-Classification-using-LSTM/blob/main/Fake_News_Classifier_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implementation Flow**

1. Dataset Loading

2. Seperation of Dependent & Independent features

3. Cleaning the data
  * Stemming
  * Stopwords removal
4. Fix the sentence length
5. One Hot Repesentation, Embedding Layer Representation
6. LSTM Neural Network
7. Bi directional LSTM Neural Network

**Dataset:**

https://www.kaggle.com/c/fake-news/data?select=train.csv

In [1]:
import pandas as pd
df = pd.read_csv('train.csv')
#df = pd.read_csv('train.csv', error_bad_lines = False, engine='python')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.loc[6][1]

'Life: Life Of Luxury: Elton John’s 6 Favorite Shark Pictures To Stare At During Long, Transcontinental Flights'

In [3]:
df.shape

(20800, 5)

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.reset_index(inplace = True)

In [7]:
df.isnull().sum()

index     0
id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
X = df.drop('label', axis = 1)

In [9]:
y = df['label']

In [10]:
X.shape

(18285, 5)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
X

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [15]:
voc_size = 5000

In [16]:
messages=X.copy()

In [17]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [18]:
messages.isnull().sum()

index     0
id        0
title     0
author    0
text      0
dtype: int64

In [19]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [20]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

[[1661, 497, 1533, 190, 4882, 2388, 2608, 2257, 511, 4178],
 [2013, 51, 2089, 2618, 335, 4209, 3533],
 [4764, 2178, 1178, 4239],
 [1536, 3367, 4601, 3154, 2753, 3943],
 [2440, 335, 4473, 4130, 4630, 4095, 335, 3754, 1212, 2682],
 [1601,
  3224,
  2746,
  1991,
  2217,
  4727,
  2647,
  143,
  1967,
  4992,
  1151,
  371,
  1932,
  2432,
  3533],
 [4640, 776, 244, 2688, 2409, 344, 2323, 3347, 4256, 1970, 1375],
 [4365, 4999, 314, 2626, 201, 1668, 4727, 402, 4256, 1970, 1375],
 [2116, 1818, 412, 4851, 4730, 2718, 609, 1033, 4727, 879],
 [3469, 4148, 709, 4179, 3774, 3658, 4031, 863],
 [4327, 3138, 2410, 2912, 2436, 4932, 2223, 1811, 3483, 4056, 316],
 [3154, 2851, 4882, 2718, 4727, 201],
 [2850, 863, 1714, 2168, 1731, 3156, 4554, 711, 2146],
 [1418, 3605, 1841, 2022, 2638, 1200, 3208, 4256, 1970, 1375],
 [1212, 1374, 3898, 675, 2621, 4256, 1970, 1375],
 [1387, 2228, 396, 1882, 4154, 1278, 3876, 595, 4369, 4707],
 [2537, 117, 51],
 [3294, 4530, 3898, 2878, 4727, 2443, 986, 3533],
 [4394, 

In [26]:
print(corpus[1])
print(onehot_rep[1])

flynn hillari clinton big woman campu breitbart
[2013, 51, 2089, 2618, 335, 4209, 3533]


**Embedding Representation**

In [27]:
sent_length = 25
embedded_docs = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 2257  511 4178]
 [   0    0    0 ...  335 4209 3533]
 [   0    0    0 ... 2178 1178 4239]
 ...
 [   0    0    0 ... 4256 1970 1375]
 [   0    0    0 ... 1719 2384 3239]
 [   0    0    0 ...  209 4934 4565]]


In [28]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0, 2013,   51, 2089, 2618,
        335, 4209, 3533], dtype=int32)

**LSTM**

In [29]:
# creating model LSTM

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))  # 100 No. of LSTM cells
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [30]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [31]:
len(embedded_docs), y.shape

(18285, (18285,))

In [32]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [33]:
X_final.shape, y_final.shape

((18285, 25), (18285,))

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.2)

**LSTM Model Training**

In [35]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
229/229 [==============================] - 19s 64ms/step - loss: 0.3200 - accuracy: 0.8515 - val_loss: 0.1777 - val_accuracy: 0.9253
Epoch 2/10
229/229 [==============================] - 3s 15ms/step - loss: 0.1444 - accuracy: 0.9422 - val_loss: 0.1819 - val_accuracy: 0.9245
Epoch 3/10
229/229 [==============================] - 3s 11ms/step - loss: 0.1046 - accuracy: 0.9612 - val_loss: 0.1961 - val_accuracy: 0.9270
Epoch 4/10
229/229 [==============================] - 2s 9ms/step - loss: 0.0726 - accuracy: 0.9753 - val_loss: 0.2209 - val_accuracy: 0.9275
Epoch 5/10
229/229 [==============================] - 3s 14ms/step - loss: 0.0469 - accuracy: 0.9839 - val_loss: 0.2528 - val_accuracy: 0.9226
Epoch 6/10
229/229 [==============================] - 3s 12ms/step - loss: 0.0291 - accuracy: 0.9916 - val_loss: 0.3103 - val_accuracy: 0.9278
Epoch 7/10
229/229 [==============================] - 2s 8ms/step - loss: 0.0205 - accuracy: 0.9945 - val_loss: 0.3250 - val_accuracy: 0.9237


**Performance Metrics and Accuracy**

In [36]:
y_pred = model.predict(X_test)

115/115 [==============================] - 1s 2ms/step


In [37]:
y_pred = np.where(y_pred > 0.5, 1, 0)   # 0.5 --> threshold value

In [38]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [39]:
confusion_matrix(y_test, y_pred)

array([[1978,  145],
       [ 174, 1360]])

In [40]:
accuracy_score(y_test, y_pred)

0.9127700300793

In [41]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      2152
           1       0.89      0.90      0.90      1505

    accuracy                           0.91      3657
   macro avg       0.91      0.91      0.91      3657
weighted avg       0.91      0.91      0.91      3657



**Bi directional LSTM**

In [42]:
from tensorflow.keras.layers import Bidirectional

In [43]:
# creating model Bidirectional LSTM

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Bidirectional(LSTM(100))) # 100 --> No. of LSTM Cells
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [44]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.2)

**Bi directional LSTM Model Training**

In [47]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 32)

Epoch 1/10
458/458 [==============================] - 18s 32ms/step - loss: 0.2632 - accuracy: 0.8772 - val_loss: 0.1885 - val_accuracy: 0.9212
Epoch 2/10
458/458 [==============================] - 6s 13ms/step - loss: 0.1392 - accuracy: 0.9471 - val_loss: 0.1854 - val_accuracy: 0.9264
Epoch 3/10
458/458 [==============================] - 4s 8ms/step - loss: 0.0880 - accuracy: 0.9685 - val_loss: 0.2198 - val_accuracy: 0.9243
Epoch 4/10
458/458 [==============================] - 4s 9ms/step - loss: 0.0565 - accuracy: 0.9815 - val_loss: 0.2682 - val_accuracy: 0.9122
Epoch 5/10
458/458 [==============================] - 5s 10ms/step - loss: 0.0364 - accuracy: 0.9883 - val_loss: 0.3095 - val_accuracy: 0.9177
Epoch 6/10
458/458 [==============================] - 4s 8ms/step - loss: 0.0232 - accuracy: 0.9934 - val_loss: 0.3516 - val_accuracy: 0.9158
Epoch 7/10
458/458 [==============================] - 4s 8ms/step - loss: 0.0162 - accuracy: 0.9958 - val_loss: 0.4109 - val_accuracy: 0.9119
Ep

In [48]:
y_pred = model.predict(X_test)

115/115 [==============================] - 1s 3ms/step


In [49]:
y_pred = np.where(y_pred > 0.5, 1, 0)

**Performance Metrics and Accuracy**

In [50]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [51]:
confusion_matrix(y_test, y_pred)

array([[1860,  212],
       [  95, 1490]])

In [52]:
accuracy_score(y_test, y_pred)

0.9160514082581351

In [53]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      2072
           1       0.88      0.94      0.91      1585

    accuracy                           0.92      3657
   macro avg       0.91      0.92      0.92      3657
weighted avg       0.92      0.92      0.92      3657

